In [1]:
import pandas as pd
from collections import defaultdict
import math
from Bio import SeqIO

In [2]:
# Đường dẫn tới tệp FASTA
fasta_file = r"E:\AMP_predict\peptides.fasta"

In [5]:
def fasta_to_dataframe(fasta_file):

    data = []
    
    # Duyệt qua từng record trong tệp FASTA
    for record in SeqIO.parse(fasta_file, "fasta"):
        # Lấy ID, mô tả, và chuỗi từ từng record
        seq_id = record.id.split()[0]
        description = record.description.split()[1]
        sequence = str(record.seq)
        
        # Thêm thông tin vào danh sách
        data.append({
            "ID": seq_id,
            "Label": description,
            "Sequence": sequence
        })
    
    # Tạo DataFrame từ danh sách dữ liệu
    df = pd.DataFrame(data)
    
    return df

In [6]:
# Gọi hàm để chuyển đổi tệp FASTA thành DataFrame
df = fasta_to_dataframe(fasta_file)

# Xem dữ liệu
df.columns

Index(['ID', 'Label', 'Sequence'], dtype='object')

In [7]:
def generate_distribution_features(sequence):
    """
    Generate a dictionary of distribution descriptors for amino acid classes in a given sequence.
    
    Parameters:
    - sequence (str): The amino acid sequence.

    Returns:
    - dict: A dictionary with distribution descriptors for various properties and classes.
    """

    # Define the amino acid classes for each property
    properties = {
        "hydrophobicity": {
            "polar": "RKEDQN",
            "neutral": "GASTPHY",
            "hydrophobic": "CLVIMFW"
        },
        "vdw_volume": {
            "small": "GASTPDC",
            "medium": "NVEQIL",
            "large": "MHKFRYW"
        },
        "polarity": {
            "polar": "EDQNKR",
            "neutral": "GASTPHY",
            "nonpolar": "CLVIMFW"
        },
        "polarizability": {
            "low": "GASDT",
            "medium": "CPNVEQIL",
            "high": "KMHFRYW"
        },
        "charge": {
            "negative": "DE",
            "neutral": "ACFGHILMNPQSTVWY",
            "positive": "KR"
        },
        "secondary_structure": {
            "helix": "EALMQKRH",
            "sheet": "VIYCWT",
            "coil": "GNPSD"
        },
        "solvent_accessibility": {
            "buried": "ALFCGIVW",
            "intermediate": "MPSTHY",
            "exposed": "DEKNQR"
        }
    }

    def calculate_distribution(amino_class):
        """
        Calculate distribution descriptors for a given amino acid class in a sequence.
        Returns a dictionary with descriptors for the first, 20%, 40%, 60%, 80% and 100% residue positions.
        """
        class_residues = [i for i, aa in enumerate(sequence) if aa in amino_class]
        length = len(sequence)
        total = len(class_residues)

        if total == 0:
            # No residues of this class in sequence
            return {"first": 0, "20%": 0, "40%": 0, "60%": 0, "80%": 0, "100%": 0}

        # Calculate descriptors based on positions in sequence
        distribution = {}
        for percentile, label in zip([0, 0.2, 0.4, 0.6, 0.8, 1.0], ["first", "20%", "40%", "60%", "80%", "100%"]):
            position = math.floor(total * percentile)
            if position >= total:  # Prevent index error
                position = total - 1
            residue_position = class_residues[position]
            distribution[label] = (residue_position + 1) / length * 100  # Convert to percentage

        return distribution

    feature_dict = defaultdict(dict)

    for prop, classes in properties.items():
        for class_name, amino_class in classes.items():
            # Calculate the distribution descriptors
            distribution = calculate_distribution(amino_class)
            for key, value in distribution.items():
                feature_name = f"{prop}_{class_name}_{key}"
                feature_dict[feature_name] = value

    return dict(feature_dict)

In [8]:
# Áp dụng hàm generate_distribution_features cho từng chuỗi trong cột 'Sequence'
features_list = df['Sequence'].apply(generate_distribution_features)

# Chuyển đổi danh sách các đặc trưng thành DataFrame
features_df = pd.DataFrame(features_list.tolist())

# Nối DataFrame gốc với các đặc trưng mới
df_with_features = pd.concat([df, features_df], axis=1)

# Lưu DataFrame kết quả vào tệp CSV
output_file = r'E:\AMP_predict\features.csv'
df_with_features.to_csv(output_file, index=False)

print(f"Kết quả đã được lưu vào {output_file}")


Kết quả đã được lưu vào E:\AMP_predict\features.csv
